In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from PIL import Image
import tensorflow_hub as hub
from tensorflow.keras.models import load_model
import cv2

In [2]:
train_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_datagen.flow_from_directory(
        '../data/dataset/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '../data/dataset/val',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle = False)
test_generator = test_datagen.flow_from_directory(
        '../data/dataset/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle = False)

Found 2437 images belonging to 2 classes.
Found 429 images belonging to 2 classes.
Found 568 images belonging to 2 classes.


In [40]:
base_model.summary()

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_graph_network_for_inferred_shape',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_los

In [31]:
def add_new_last_layer(base_model, nb_classes=2):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    # Get the output shape of the models last layer
    x = base_model.output
    # construct new head for the transferred model
    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten(name="flatten")(x) 
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [32]:
# This will freeze the weights on all the layers except for our new dense layer
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
train_model = add_new_last_layer(base_model)
setup_to_transfer_learn(train_model,base_model)

In [34]:
for i, layer in enumerate(train_model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 Conv1_pad False
2 Conv1 False
3 bn_Conv1 False
4 Conv1_relu False
5 expanded_conv_depthwise False
6 expanded_conv_depthwise_BN False
7 expanded_conv_depthwise_relu False
8 expanded_conv_project False
9 expanded_conv_project_BN False
10 block_1_expand False
11 block_1_expand_BN False
12 block_1_expand_relu False
13 block_1_pad False
14 block_1_depthwise False
15 block_1_depthwise_BN False
16 block_1_depthwise_relu False
17 block_1_project False
18 block_1_project_BN False
19 block_2_expand False
20 block_2_expand_BN False
21 block_2_expand_relu False
22 block_2_depthwise False
23 block_2_depthwise_BN False
24 block_2_depthwise_relu False
25 block_2_project False
26 block_2_project_BN False
27 block_2_add False
28 block_3_expand False
29 block_3_expand_BN False
30 block_3_expand_relu False
31 block_3_pad False
32 block_3_depthwise False
33 block_3_depthwise_BN False
34 block_3_depthwise_relu False
35 block_3_project False
36 block_3_project_BN False
37 block_4_expand Fa

In [41]:
# Set up Tensorboard for logging models
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir(root_logdir):
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir(root_logdir)
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

In [70]:
# Set up checkpoints 
import time
timestamp = time.strftime('%Y_%m_%d-%H_%M_%S')
checkpoint_filepath = './mobilenetv2_no_ssd_' + timestamp + '.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

In [71]:
init_lr = 1e-4
n_epochs = 20
batchsize = 32
opt = Adam(lr=init_lr, decay=init_lr / n_epochs)

In [12]:
tf.saved_model.load('saved_model.pb')

OSError: SavedModel file does not exist at: saved_model.pb/{saved_model.pbtxt|saved_model.pb}

In [13]:
!ls
    

builders
face_detector
mobilenetv2_mod_adam_2021_01_06-00_01_24.h5
mobilenetv2_no_ssd_2021_01_05-21_43_31.h5
my_logs
object_detection
preprocessing.py
saved_model.pb
single_mask_detector_model.h5
ssd_mobilenet_v2_320x320_coco17_tpu-8
training_mobilenetv2_def_adam.py
training_mobilenetv2_mod_adam.py
transfer_learning_model-mobilenetv2SSD.ipynb
transfer_learning_model.ipynb
utils
variables
webcam_dnn_test.py
webcam_test.py


In [68]:
#test if model works before placing into 'training_script.py'
train_model.fit(
    train_generator,
    epochs= 1,
#     steps_per_epoch= 2437 // batchsize,
    validation_data= validation_generator,
#     validation_steps= 568 // batchsize,
    callbacks = [checkpoint_cb,early_stopping_cb,tensorboard_cb])

76/76 [==============================] - ETA: 0s - loss: 0.0607 - accuracy: 0.9780

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


76/76 [==============================] - 39s 511ms/step - loss: 0.0607 - accuracy: 0.9780 - val_loss: 0.0510 - val_accuracy: 0.9790
